In [76]:
import logging
from itertools import combinations
import pandas as pd
from tqdm.auto import tqdm

import random

import numpy as np
from geopy.distance import geodesic
import networkx as nx

from icecream import ic

logging.basicConfig(level=logging.DEBUG)

In [77]:
CITIES = pd.read_csv('~/Computational_Intelligence/labs/lab2/cities/vanuatu.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))

for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Isangel,-19.53,169.28
1,Lakatoro,-16.09,167.40
2,Longana,-15.30,168.00
3,Luganville,-15.51,167.15
4,Norsup,-16.07,167.39


In [78]:
def print_solution(solution):
    for c1, c2 in zip(solution, solution[1:]):
        logging.info(
            f"step: {CITIES.at[c1,'name']} -> {CITIES.at[c2,'name']} ({DIST_MATRIX[c1,c2]:.2f}km)"
        )

In [79]:
def fitness(solution):

    tot_cost = 0
    for c1, c2 in zip(solution, solution[1:]):
        tot_cost += DIST_MATRIX[c1, c2]

    # tot_cost += DIST_MATRIX[solution[0], solution[-1]]
    return tot_cost

#  mutazione 
def insert_mutation(path):
    path = path.copy()
    i, j = random.sample(range(1, len(path) - 1), 2 )   

    if i < j:
        element = path.pop(j)
        path.insert(i + 1, element)
    else:
        element = path.pop(j)
        path.insert(i, element)

    return path

def insert_mutation_temp(path, temperature):
    path = path.copy()
    
    i = random.randint(1, len(path) - 2)
    
    max_distance = int((temperature / 1000) * (len(path) - 2))
    max_distance = max(1, min(max_distance, len(path) - 2))  

    if random.random() < 0.5:
        j = max(1, i - random.randint(1, max_distance))
    else:
        j = min(len(path) - 2, i + random.randint(1, max_distance))

    if i < j:
        element = path.pop(j)
        path.insert(i + 1, element)
    else:
        element = path.pop(j)
        path.insert(i, element)

    return path 

In [80]:
initial_temp = 1000  
cooling_rate = 0.995

# buffer_size = int(0.02 * len(CITIES))    # change this number to 4 for smaller countries like italy
buffer_size = 4

current_path = list(range(len(CITIES))) + [0]    


best_path = current_path
current_cost = fitness(current_path) 
best_cost = current_cost

temperature = initial_temp
cost_buffer = []

for iteration in tqdm (range(500000) ): 
        
    # new_path = insert_mutation(current_path)  
    new_path = insert_mutation_temp(current_path, temperature)  

    new_cost = fitness(new_path)

    delta_cost = new_cost - current_cost

    if delta_cost < 0 or np.random.random() < np.exp(-delta_cost / temperature):
        current_path = new_path
        current_cost = new_cost

        if current_cost < best_cost:
            best_cost = current_cost
            best_path = current_path

    
    cost_buffer.append(current_cost)
    if len(cost_buffer) > buffer_size:
        cost_buffer.pop(0)

    if iteration % buffer_size == 0 and len(cost_buffer) == buffer_size:
        avg_cost = np.mean(cost_buffer)
        if avg_cost < current_cost: 
            temperature *= cooling_rate


    if iteration % 10000 == 0:
        logging.info(f"Iteration {iteration}, Best Cost: {best_cost:.2f} km, Temperature: {temperature:.2f}")

    if temperature < 1e-5:
        break

print_solution(best_path)
logging.info(f"Best path found with length {best_cost:.2f} km,  after {iteration} iterations")


  0%|          | 0/500000 [00:00<?, ?it/s]

INFO:root:Iteration 0, Best Cost: 1618.29 km, Temperature: 1000.00


INFO:root:Iteration 10000, Best Cost: 1345.54 km, Temperature: 28.05
INFO:root:Iteration 20000, Best Cost: 1345.54 km, Temperature: 8.38
INFO:root:Iteration 30000, Best Cost: 1345.54 km, Temperature: 3.47
INFO:root:Iteration 40000, Best Cost: 1345.54 km, Temperature: 2.04
INFO:root:Iteration 50000, Best Cost: 1345.54 km, Temperature: 1.58
INFO:root:Iteration 60000, Best Cost: 1345.54 km, Temperature: 1.26
INFO:root:Iteration 70000, Best Cost: 1345.54 km, Temperature: 1.11
INFO:root:Iteration 80000, Best Cost: 1345.54 km, Temperature: 1.02
INFO:root:Iteration 90000, Best Cost: 1345.54 km, Temperature: 0.96
INFO:root:Iteration 100000, Best Cost: 1345.54 km, Temperature: 0.91
INFO:root:Iteration 110000, Best Cost: 1345.54 km, Temperature: 0.88
INFO:root:Iteration 120000, Best Cost: 1345.54 km, Temperature: 0.88
INFO:root:Iteration 130000, Best Cost: 1345.54 km, Temperature: 0.86
INFO:root:Iteration 140000, Best Cost: 1345.54 km, Temperature: 0.84
INFO:root:Iteration 150000, Best Cost: 134